# Necessary Steps

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/MediaCloud

/content/drive/MyDrive/MediaCloud


In [ ]:
import os
pwd = os.getcwd()
pwd

'/content/drive/MyDrive/MediaCloud'

# Install Libraries

In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes
!pip install sentencepiece
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

# Hugging Face and Libraries

In [ ]:
pwd

'/content/drive/MyDrive/MediaCloud'

In [ ]:
from huggingface_hub import login

# Use your Hugging Face access token (get it from https://huggingface.co/settings/tokens)
login('ENTER TOKEN HERE')

In [ ]:
import torch
import transformers
from transformers import AutoTokenizer
from  langchain import LLMChain, HuggingFacePipeline, PromptTemplate
import pandas as pd
import os

# Model

In [ ]:
model = "meta-llama/Llama-3.2-1B-Instruct"
#model = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    #max_length=4096,
    max_new_tokens=512,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
#    truncation=True
)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

<ipython-input-10-9410ce7de563>:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})


# Output all at once

In [ ]:
def assign_cluster(title, llm):
    template = f"""
            Given the title and first paragraph of an article (delimited by triple
            backticks), create an appropriate cluster that best fits the overall
            theme of the article.
            - First, generate a concise and meaningful cluster name based on
              the title and text.
            - Then assign the article to that single cluster.
            Please ensure that the title is classified into only one of the clusters.
            “‘title“‘
            “‘first paragraph“‘
            CLUSTER:
        """
    prompt_template = PromptTemplate(input_variables=["title"], template=template)
    llm_chain = LLMChain(prompt=prompt_template, llm=llm)

    cluster = llm_chain.run({"title": title}).strip()
    return cluster

In [ ]:
# Input and output folder paths
input_folder = 'Farmers_Protest/4.UPDATED_Parsed_YearWise_Data_with_Summaries'
output_folder = 'Farmers_Protest/5.Clustered_Articles'
os.makedirs(output_folder, exist_ok=True)

In [ ]:
# Loop through all CSV files in the input folder
for file_name in os.listdir(input_folder):
    if file_name.endswith(".csv"):  # Only process CSV files
        input_file = os.path.join(input_folder, file_name)

        # Read the CSV file
        df = pd.read_csv(input_file)

        # Ensure required columns exist
        if not {'title', 'parsed_text', 'parsed_summary'}.issubset(df.columns):
            print(f"Skipping {file_name} as it doesn't contain required columns.")
            continue

        # Keep only the desired columns
        df = df[['title', 'parsed_text', 'parsed_summary']]

        # Apply clustering to each article title
        df['cluster'] = df['title'].apply(lambda title: assign_cluster(title, llm))

        # Save the processed DataFrame to the output folder
        output_file = os.path.join(output_folder, f"Clustered_{file_name}")
        df.to_csv(output_file, index=False)

        print(f"Processed and saved: {output_file}")

print("All files processed successfully.")

<ipython-input-11-a84f367b46eb>:16: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(prompt=prompt_template, llm=llm)
<ipython-input-11-a84f367b46eb>:18: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  cluster = llm_chain.run({"title": title}).strip()
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Processed and saved: Farmers_Protest/5.Clustered_Articles/Clustered_Parsed_data_2024_with_summaries.csv
Processed and saved: Farmers_Protest/5.Clustered_Articles/Clustered_Parsed_data_2022_with_summaries.csv
Processed and saved: Farmers_Protest/5.Clustered_Articles/Clustered_Parsed_data_2021_with_summaries.csv
Processed and saved: Farmers_Protest/5.Clustered_Articles/Clustered_Parsed_data_2020_with_summaries.csv
All files processed successfully.


# Extract Cluster

In [ ]:
pwd

'/content/drive/MyDrive/MediaCloud'

In [ ]:
import pandas as pd
import re
import os
from glob import glob

In [ ]:
input_folder = 'Farmers_Protest/5.Clustered_Articles'
output_folder = 'Farmers_Protest/6.UPDATED_Clustered_Articles'
os.makedirs(output_folder, exist_ok=True)

In [ ]:
# Define a function to replace the entire text with the extracted cluster
def replace_with_cluster(article_text):
    """
    Replace the entire text with the cluster value extracted after the first 'CLUSTER:' keyword.
    """
    # Regex to match the word 'CLUSTER:' and capture the text after it
    pattern = r"CLUSTER:\s*(.*?)(?:\n|$)"
    match = re.search(pattern, article_text)
    if match:
        return match.group(1).strip()  # Extract the cluster name
    return article_text  # If no match is found, keep the original text

# Process each CSV file in the input folder
csv_files = glob(os.path.join(input_folder, '*.csv'))  # Get all CSV files in the folder

for file_path in csv_files:
    # Load the CSV file
    df = pd.read_csv(file_path)

    # Apply the function to the 'cluster' column to replace its content
    if 'cluster' in df.columns:  # Ensure the 'cluster' column exists
        df['cluster'] = df['cluster'].apply(replace_with_cluster)

        # Save the updated dataframe to a new CSV file in the output folder
        file_name = os.path.basename(file_path)  # Get the file name
        output_csv_path = os.path.join(output_folder, file_name)
        df.to_csv(output_csv_path, index=False)

        print(f"Processed and saved: {output_csv_path}")
    else:
        print(f"Skipping {file_path}: 'cluster' column not found.")

Processed and saved: Farmers_Protest/6.UPDATED_Clustered_Articles/Clustered_Parsed_data_2024_with_summaries.csv
Processed and saved: Farmers_Protest/6.UPDATED_Clustered_Articles/Clustered_Parsed_data_2022_with_summaries.csv
Processed and saved: Farmers_Protest/6.UPDATED_Clustered_Articles/Clustered_Parsed_data_2021_with_summaries.csv
Processed and saved: Farmers_Protest/6.UPDATED_Clustered_Articles/Clustered_Parsed_data_2020_with_summaries.csv


# Separate based on similar clusters

### SPECIFY YEAR

In [ ]:
import os
import pandas as pd


In [ ]:
df = pd.read_csv('Farmers_Protest/6.UPDATED_Clustered_Articles/Clustered_Parsed_data_2024_with_summaries.csv')

In [ ]:
output_folder = 'Farmers_Protest/7.Separate_Clusters/2024'  # Specify the folder to save the files
os.makedirs(output_folder, exist_ok=True)


In [ ]:
unique_clusters = df['cluster'].unique()


In [ ]:
# Iterate through each cluster and save the corresponding rows to a CSV
for cluster in unique_clusters:
    # Filter the DataFrame for the current cluster
    cluster_data = df[df['cluster'] == cluster]

    # Replace problematic characters in the file name (e.g., slashes, spaces)
    safe_cluster_name = cluster.replace(' ', '_').replace('/', '_')

    # Create a file name using the cluster name
    output_file = os.path.join(output_folder, f"{safe_cluster_name}.csv")

    # Save the filtered data to a CSV file
    cluster_data.to_csv(output_file, index=False)

print(f"CSV files saved in {output_folder}")

CSV files saved in Farmers_Protest/7.Separate_Clusters/2024
